## 词向量训练

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
from tqdm.auto import tqdm

#### 数据为：NLTK 中提供的 Reuters 语料库

在训练语言模型的过程中需要先定义一些变量，比如句首标记、句尾标记、以及构建批次时用于补齐序列长度的标记

In [2]:
# Constants
BOS_TOKEN = "<bos>" ## 句首标记
EOS_TOKEN = "<eos>" ## 句尾标记
PAD_TOKEN = "<pad>" ## 补齐标记

加载 Reuters 语料库并构建数据集，同时建立词表，这里需要用到前面的vocab

#### 1、词表映射

In [3]:
from collections import defaultdict, Counter

class Vocab:
    def __init__(self, tokens=None):
        self.idx_to_token = list()
        self.token_to_idx = dict()

        if tokens is not None:
            if "<unk>" not in tokens:
                tokens = tokens + ["<unk>"]
            for token in tokens:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1
            self.unk = self.token_to_idx['<unk>']

    @classmethod
    def build(cls, text, min_freq=1, reserved_tokens=None):
        token_freqs = defaultdict(int)
        for sentence in text:
            for token in sentence:
                token_freqs[token] += 1
        uniq_tokens = ["<unk>"] + (reserved_tokens if reserved_tokens else [])
        uniq_tokens += [token for token, freq in token_freqs.items() \
                        if freq >= min_freq and token != "<unk>"]
        
        return cls(uniq_tokens)   ## 返回cls 对象，到时候就可以通过这个cls 来调用 Vocab 类中的其他方法
                                  ## 后面调用 build ，并且将返回值设为 vocab = cls(uniq_tokens) ，之后调用vocab，会默认带有uniq_tokens参数
                                 ## 此 uniq_tokens 参数会传给 __init__ 中的 tokens。
    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, token):
        return self.token_to_idx.get(token, self.unk)

    def convert_tokens_to_ids(self, tokens):
        return [self[token] for token in tokens] 

    def convert_ids_to_tokens(self, indices):
        return [self.idx_to_token[index] for index in indices]

#### 下载数据

In [ ]:
import nltk
nltk.download('reuters')
nltk.download('punkt')

#### 加载数据

In [4]:
from nltk.corpus import reuters

reuters.sents()

[['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN', 'RIFT', 'Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'Asia', "'", 's', 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far', '-', 'reaching', 'economic', 'damage', ',', 'businessmen', 'and', 'officials', 'said', '.'], ['They', 'told', 'Reuter', 'correspondents', 'in', 'Asian', 'capitals', 'a', 'U', '.', 'S', '.', 'Move', 'against', 'Japan', 'might', 'boost', 'protectionist', 'sentiment', 'in', 'the', 'U', '.', 'S', '.', 'And', 'lead', 'to', 'curbs', 'on', 'American', 'imports', 'of', 'their', 'products', '.'], ...]

In [5]:
# len(reuters.sents()) ## 54716

In [5]:
def load_reuters():
    text = reuters.sents()[:1000]
    # lowercase (optional)
    text = [[word.lower() for word in sentence] for sentence in text]
    vocab = Vocab.build(text, reserved_tokens=[PAD_TOKEN, BOS_TOKEN, EOS_TOKEN])
    corpus = [vocab.convert_tokens_to_ids(sentence) for sentence in text]

    return corpus, vocab

In [7]:
corpus, vocab = load_reuters()

#### 创建前馈神经网络语言模型处理类

In [ ]:
from torch.utils.data import Dataset

class NGramDataset(Dataset):
    def __init__(self, corpus, vocab, context_size=2):
        self.data = []
        self.bos = vocab[BOS_TOKEN]
        self.eos = vocab[EOS_TOKEN]
        for sentence in tqdm(corpus, desc="Dataset Construction"):
            # 插入句首句尾符号
            sentence = [self.bos] + sentence + [self.eos]
            if len(sentence) < context_size:
                continue
            for i in range(context_size, len(sentence)):
                # 模型输入：长为context_size的上文
                context = sentence[i-context_size:i]
                # 模型输出：当前词
                target = sentence[i]
                self.data.append((context, target))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        print('调用了我')
        return self.data[i]

    def collate_fn(self, examples):
        # 从独立样本集合中构建batch输入输出
        inputs = torch.tensor([ex[0] for ex in examples], dtype=torch.long)
        targets = torch.tensor([ex[1] for ex in examples], dtype=torch.long)
        return (inputs, targets)

In [ ]:
## 设置权重
# def init_weights(model):
#     for name, param in model.named_parameters():
#         if "embedding" not in name:
#             torch.nn.init.uniform_(
#                 param, a=-WEIGHT_INIT_RANGE, b=WEIGHT_INIT_RANGE
#             )

#### 创建前馈神经网络语言模型

词向量层中的向量会随着模型的优化进行更新

In [ ]:
import torch
from torch import nn, optim
import torch.nn.functional as F

class FeedForwardNNLM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size, hidden_dim):
        super(FeedForwardNNLM, self).__init__()
        # 词嵌入层
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        # 线性变换：词嵌入层->隐含层
        self.linear1 = nn.Linear(context_size * embedding_dim, hidden_dim)
        # 线性变换：隐含层->输出层
        self.linear2 = nn.Linear(hidden_dim, vocab_size) ## 最后输出神经元数量为词表大小数量，最后需要预测下一个词可能出现的概率
        # 使用ReLU激活函数
        self.activate = F.relu
#         init_weights(self)

    def forward(self, inputs):
        ## embeddings(inputs).shape 为 torch.Size([1024, 2, 64]) ，embeds shape 为 torch.Size([1024, 128]) 
        ## 所以上面隐藏层输入神经元维度为 context_size * embedding_dim 
        embeds = self.embeddings(inputs).view((inputs.shape[0], -1))
        hidden = self.activate(self.linear1(embeds))
        output = self.linear2(hidden)
        # 根据输出层（logits）计算概率分布并取对数，以便于计算对数似然
        # 这里采用PyTorch库的log_softmax实现
        log_probs = F.log_softmax(output, dim=1)
        return log_probs

#### 将数据转为 dataloader 格式

In [6]:
from torch.utils.data import Dataset, DataLoader

def get_loader(dataset, batch_size, shuffle=True):
    data_loader = DataLoader(
        dataset,
        batch_size=batch_size,
        collate_fn=dataset.collate_fn,
        shuffle=shuffle
    )
    return data_loader

In [ ]:
from tqdm.auto import tqdm

## 定义超参数
embedding_dim = 64
context_size = 2
hidden_dim = 128
batch_size = 1024
num_epoch = 10

# 读取文本数据，构建FFNNLM训练数据集（n-grams）
corpus, vocab = load_reuters()                  ## corpus 是将 文字 转 idx 之后得到的值。

## 为了给 dataLoader 提供dataset 参数，第一步需要先将数据传到 NGramDataset 中
dataset = NGramDataset(corpus, vocab, context_size)
## 然后再将 dataset 作为参数传到 get_loader 中，创建dataLoader对象
data_loader = get_loader(dataset, batch_size)

In [ ]:
corpus[1]

In [ ]:
# 负对数似然损失函数
nll_loss = nn.NLLLoss()
# 构建FFNNLM，并加载至device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = FeedForwardNNLM(len(vocab), embedding_dim, context_size, hidden_dim)
model.to(device)

In [ ]:
# 使用Adam优化器
optimizer = optim.Adam(model.parameters(), lr=0.001)

model.train()
total_losses = []
for epoch in range(num_epoch):
    total_loss = 0
    for batch in tqdm(data_loader, desc=f"Training Epoch {epoch}"):
        inputs, targets = [x.to(device) for x in batch]
        optimizer.zero_grad()
        log_probs = model(inputs)
        loss = nll_loss(log_probs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Loss: {total_loss:.2f}")
    total_losses.append(total_loss)

In [12]:
def save_pretrained(vocab, embeds, save_path):
    """
    Save pretrained token vectors in a unified format, where the first line
    specifies the `number_of_tokens` and `embedding_dim` followed with all
    token vectors, one token per line.
    """
    with open(save_path, "w") as writer:
        writer.write(f"{embeds.shape[0]} {embeds.shape[1]}\n")
        for idx, token in enumerate(vocab.idx_to_token):
            vec = " ".join(["{:.4f}".format(x) for x in embeds[idx]])
            writer.write(f"{token} {vec}\n")
    print(f"Pretrained embeddings saved to: {save_path}")

In [ ]:
# 保存词向量（model.embeddings）
save_pretrained(vocab, model.embeddings.weight.data, "ffnnlm.vec")

## ------------------------------------------------------- 调试部分 ------------------------------------------------------------ 

In [ ]:
for batch in tqdm(data_loader, desc=f"Training Epoch {epoch}"):
    inputs, targets = [x for x in batch]
    break

In [ ]:
embeddings = nn.Embedding(31081, 64)
embeds = embeddings(inputs).view((inputs.shape[0], -1))

In [ ]:
embeddings(inputs).shape

In [ ]:
embeds.shape

In [ ]:
inputs[2]

In [ ]:
targets[2]

# RNNNLM

In [7]:
from torch.nn.utils.rnn import pad_sequence


class RnnlmDataset(Dataset):
    def __init__(self, corpus, vocab):
        self.data = []
        self.bos = vocab[BOS_TOKEN]
        self.eos = vocab[EOS_TOKEN]
        self.pad = vocab[PAD_TOKEN]
        for sentence in tqdm(corpus, desc="Dataset Construction"):
            # 模型输入：BOS_TOKEN, w_1, w_2, ..., w_n
            input = [self.bos] + sentence
            # 模型输出：w_1, w_2, ..., w_n, EOS_TOKEN
            target = sentence + [self.eos]
            self.data.append((input, target))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        return self.data[i]

    def collate_fn(self, examples):
        # 从独立样本集合中构建batch输入输出
        inputs = [torch.tensor(ex[0]) for ex in examples]
        targets = [torch.tensor(ex[1]) for ex in examples]
        # 对batch内的样本进行padding，使其具有相同长度
        inputs = pad_sequence(inputs, batch_first=True, padding_value=self.pad)
        targets = pad_sequence(targets, batch_first=True, padding_value=self.pad)
        return (inputs, targets)

In [8]:
class RNNLM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(RNNLM, self).__init__()
        # 词嵌入层
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        # 循环神经网络：这里使用LSTM
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        # 输出层
        self.output = nn.Linear(hidden_dim, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        # 计算每一时刻的隐含层表示
        hidden, _ = self.rnn(embeds)
        output = self.output(hidden)
        log_probs = F.log_softmax(output, dim=2)
        return log_probs

In [9]:
embedding_dim = 64
hidden_dim = 128
batch_size = 64
num_epoch = 500

# 读取文本数据，构建FFNNLM训练数据集（n-grams）
corpus, vocab = load_reuters()
dataset = RnnlmDataset(corpus, vocab)
data_loader = get_loader(dataset, batch_size)

HBox(children=(IntProgress(value=0, description='Dataset Construction', max=1000, style=ProgressStyle(description_width='initial')), HTML(value='')))

In [11]:
for batch in tqdm(data_loader, desc=f"Training Epoch {epoch}"):
    inputs, targets = [x for x in batch]
    break

NameError: name 'epoch' is not defined

In [45]:
inputs.shape

torch.Size([64, 71])

In [46]:
targets.view(-1).shape

torch.Size([4544])

In [47]:
inputs[0]

tensor([   2,  121, 1715,  167, 1716,  211,  102,   19, 1717,   47,   19,   16,
         910,   39,   64,  122,  126,   97, 1718,  228,  211,   57, 1719, 1720,
           8, 1489, 1721,   26, 1722,   31,  328,   57, 1723,   59, 1131,   20,
        1698, 1311,  128,  283,   42,   10,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1])

In [48]:
' '.join(vocab.convert_ids_to_tokens(inputs[0]))

'<bos> " there are obvious problems at the moment in the trade area , but we do not wish those problems to divert attention from important areas of cooperation that continue to exist on security and political issues ," he said . <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>'

In [49]:
targets[0]

tensor([ 121, 1715,  167, 1716,  211,  102,   19, 1717,   47,   19,   16,  910,
          39,   64,  122,  126,   97, 1718,  228,  211,   57, 1719, 1720,    8,
        1489, 1721,   26, 1722,   31,  328,   57, 1723,   59, 1131,   20, 1698,
        1311,  128,  283,   42,   10,    3,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1])

In [50]:
torch.unsqueeze(inputs[6],dim=0)

tensor([[  2, 121, 122, 989,  57, 990, 119, 127, 991, 128, 892,  42,  10,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1]])

In [51]:
' '.join(vocab.convert_ids_to_tokens(dataset[1][0]))

'<bos> they told reuter correspondents in asian capitals a u . s . move against japan might boost protectionist sentiment in the u . s . and lead to curbs on american imports of their products .'

In [34]:
' '.join(vocab.convert_ids_to_tokens(dataset[1][1]))

'they told reuter correspondents in asian capitals a u . s . move against japan might boost protectionist sentiment in the u . s . and lead to curbs on american imports of their products . <eos>'

In [10]:
# 负对数似然损失函数，忽略pad_token处的损失
## 在使用nllloss时，需要有两个张量，一个是预测向量，一个是label
nll_loss = nn.NLLLoss(ignore_index=dataset.pad)
# 构建RNNLM，并加载至device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RNNLM(len(vocab), embedding_dim, hidden_dim)
model.to(device)

RNNLM(
  (embeddings): Embedding(4168, 64)
  (rnn): LSTM(64, 128, batch_first=True)
  (output): Linear(in_features=128, out_features=4168, bias=True)
)

In [11]:
# 使用Adam优化器
optimizer = optim.Adam(model.parameters(), lr=0.001)

model.train()
for epoch in range(num_epoch):
    total_loss = 0
    for batch in tqdm(data_loader, desc=f"Training Epoch {epoch}"):
        inputs, targets = [x.to(device) for x in batch]
        optimizer.zero_grad()
        log_probs = model(inputs)
#         print(log_probs.view(-1, log_probs.shape[-1]).shape)
        loss = nll_loss(log_probs.view(-1, log_probs.shape[-1]), targets.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Loss: {total_loss:.2f}")

# save_pretrained(vocab, model.embeddings.weight.data, "rnnlm.vec")

HBox(children=(IntProgress(value=0, description='Training Epoch 0', max=16, style=ProgressStyle(description_width='initial')), HTML(value='')))


Loss: 131.98


HBox(children=(IntProgress(value=0, description='Training Epoch 1', max=16, style=ProgressStyle(description_width='initial')), HTML(value='')))


Loss: 114.35


HBox(children=(IntProgress(value=0, description='Training Epoch 2', max=16, style=ProgressStyle(description_width='initial')), HTML(value='')))

...


Loss: 4.16


HBox(children=(IntProgress(value=0, description='Training Epoch 498', max=16, style=ProgressStyle(description_width='initial')), HTML(value='')))


Loss: 4.13


HBox(children=(IntProgress(value=0, description='Training Epoch 499', max=16, style=ProgressStyle(description_width='initial')), HTML(value='')))


Loss: 4.14


In [15]:
def load_pretrained(load_path):
    with open(load_path, "r") as fin:
        # Optional: depending on the specific format of pretrained vector file
        n, d = map(int, fin.readline().split())
        tokens = []
        embeds = []
        for line in fin:
            line = line.rstrip().split(' ')
            token, embed = line[0], list(map(float, line[1:]))
            tokens.append(token)
            embeds.append(embed)
        vocab = Vocab(tokens)
        embeds = torch.tensor(embeds, dtype=torch.float)
    return vocab, embeds

In [16]:
vocab,embeds = load_pretrained('./rnnlm.vec')

In [23]:
embeds

tensor([[ 2.1060, -0.9033,  0.7795,  ..., -0.0314,  0.7060, -2.9022],
        [ 0.4234, -0.6294, -0.7711,  ..., -0.4676,  3.0436,  0.0335],
        [ 0.7837,  1.4697, -0.7131,  ...,  0.2140,  0.9701, -0.5033],
        ...,
        [-1.2039, -0.8122,  0.6347,  ..., -0.0931,  0.3163,  0.0917],
        [ 0.5013, -0.4576,  0.6905,  ...,  1.6615, -1.2897,  1.1877],
        [ 1.3486,  0.6406,  0.1452,  ..., -1.5763, -0.8749, -0.0463]])

In [20]:
class RNNLM2(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim,embeds):
        super(RNNLM2, self).__init__()
        # 词嵌入层
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.embeddings.weight.data.copy_(embeds)
        self.embeddings.weight.requires_grad = False
        # 循环神经网络：这里使用LSTM
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        # 输出层
        self.output = nn.Linear(hidden_dim, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        # 计算每一时刻的隐含层表示
        hidden, _ = self.rnn(embeds)
        output = self.output(hidden)
        log_probs = F.log_softmax(output, dim=2)
        return log_probs

In [21]:
# 负对数似然损失函数，忽略pad_token处的损失
## 在使用nllloss时，需要有两个张量，一个是预测向量，一个是label
nll_loss = nn.NLLLoss(ignore_index=dataset.pad)
# 构建RNNLM，并加载至device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model2 = RNNLM2(len(vocab), embedding_dim, hidden_dim,embeds)
model2.to(device)

RNNLM2(
  (embeddings): Embedding(4168, 64)
  (rnn): LSTM(64, 128, batch_first=True)
  (output): Linear(in_features=128, out_features=4168, bias=True)
)

In [24]:
# 使用Adam优化器
optimizer = optim.Adam(model2.parameters(), lr=0.001)

model2.train()
for epoch in range(num_epoch):
    total_loss = 0
    for batch in tqdm(data_loader, desc=f"Training Epoch {epoch}"):
        inputs, targets = [x.to(device) for x in batch]
        optimizer.zero_grad()
        log_probs = model2(inputs)
        loss = nll_loss(log_probs.view(-1, log_probs.shape[-1]), targets.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Loss: {total_loss:.2f}")

# save_pretrained(vocab, model.embeddings.weight.data, "rnnlm.vec")

HBox(children=(IntProgress(value=0, description='Training Epoch 0', max=16, style=ProgressStyle(description_width='initial')), HTML(value='')))


Loss: 131.81


HBox(children=(IntProgress(value=0, description='Training Epoch 1', max=16, style=ProgressStyle(description_width='initial')), HTML(value='')))


Loss: 116.11


HBox(children=(IntProgress(value=0, description='Training Epoch 2', max=16, style=ProgressStyle(description_width='initial')), HTML(value='')))

...


Loss: 35.72


HBox(children=(IntProgress(value=0, description='Training Epoch 98', max=16, style=ProgressStyle(description_width='initial')), HTML(value='')))


Loss: 35.40


HBox(children=(IntProgress(value=0, description='Training Epoch 99', max=16, style=ProgressStyle(description_width='initial')), HTML(value='')))


Loss: 35.09


In [39]:
' '.join(vocab.convert_ids_to_tokens(targets[30]))

'marble financial corp & lt ; mrbl > 1st qtr net oper shr 26 cts vs not given oper net 866 , 000 vs 480 , 000 note : 1987 net excludes 157 , 000 dlr gain from termination of pension plan . <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>'

In [23]:
a = targets[30][:10]

In [27]:
inputs.shape

torch.Size([40, 63])

In [28]:
inputs = inputs.to(torch.device('cuda'))

In [24]:
with torch.no_grad():
    output = model(torch.unsqueeze(a,dim=0))


In [ ]:
with torch.no_grad():
    output2 = model2(torch.unsqueeze(inputs[30],dim=0))

In [25]:
output.shape

torch.Size([1, 10, 4168])

In [26]:
targets[30]

tensor([1561,   39,  114, 1428,  112,    9,   10,   11,   10, 2518,  252,  249,
        3545,   59,  352, 1512, 2869, 1171,  363,   57, 3546, 3547, 2240,   39,
        3548,   19, 3549,   20, 3550, 3494, 1032,   10,    3,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1], device='cuda:0')

In [27]:
output.argmax(dim=2)

tensor([[2069,   39,  519,  830,    9,   10,   11,   10, 3483,  116]],
       device='cuda:0')

In [17]:
# output2.argmax(dim=2)

In [28]:
output.view(-1, output.shape[-1]).shape

torch.Size([10, 4168])

In [29]:
targets.view(-1).shape

torch.Size([2480])

In [30]:
vocab.convert_ids_to_tokens(torch.squeeze(output.argmax(dim=2)))

['form',
 ',',
 'prices',
 'about',
 'u',
 '.',
 's',
 '.',
 'petrochemical',
 'exports']

In [35]:
vocab.convert_ids_to_tokens(torch.squeeze(output2.argmax(dim=2)))

['the',
 'financial',
 'corp',
 '&',
 'lt',
 ';',
 'rbd',
 '>',
 '1st',
 'qtr',
 'net',
 'shr',
 'shr',
 '1',
 'cts',
 'vs',
 'loss',
 'given',
 'net',
 'net',
 '866',
 ',',
 '000',
 'vs',
 'loss',
 ',',
 '000',
 'revs',
 ':',
 '1987',
 'net',
 'excludes',
 '157',
 ',',
 '000',
 'dlrs',
 ',',
 'of',
 'termination',
 'of',
 'pension',
 '.',
 '.',
 '<eos>',
 'mln',
 'to',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.']

In [ ]:
inputs[1]

In [60]:
targets[30]

tensor([  19,  448,   57,  206, 3191,  284, 1555,  118,   19, 4157,  589, 4164,
         192,   19, 1749, 3696, 4156,  287,   39,   19,  129,   42,   10,    3,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1])

## ------------------------------------------------------- 调试部分 ------------------------------------------------------------ 

In [ ]:
data = []
bos = vocab[BOS_TOKEN]
eos = vocab[EOS_TOKEN]
pad = vocab[PAD_TOKEN]
for sentence in tqdm(corpus, desc="Dataset Construction"):
    # 模型输入：BOS_TOKEN, w_1, w_2, ..., w_n
    input = ['bos'] + sentence
    # 模型输出：w_1, w_2, ..., w_n, EOS_TOKEN
    target = sentence + ['eos']
    data.append((input, target))
    break